In [1]:
import os

import bids

from bids import BIDSLayout
from bids import BIDSValidator

# BrainVoyager python interface
import bvbabel

import pandas as pd



In [2]:
# Function to convert prt to tsv
def convert_prt_to_tsv(prt_file):
    """
    Convert BrainVoyager PRT file to BIDS TSV file.
    """

    # -- Create Dataframe with duration, onset and trial_Type ---
    bids_df = pd.DataFrame(columns = ['onset', 'duration','trial_type'])

    # -- Read .prt file --
    header, data = bvbabel.prt.read_prt(prt_file)

    # Print header information
    print("\nPRT header")
    for key, value in header.items():
        print("  ", key, ":", value)

    # Print data
    print("\nPRT data")
    for d in data:
        for key, value in d.items():
            print("  ", key, ":", value)
        print("")

    # -- Get duration, onset and trial_type and add to bids_df --
    for d in data:
        for key, value in d.items():
            # iterate over 'Time start' key
            if key == 'Time start':
                print(d)
                # add value to trial_data
                bids_df = pd.concat([bids_df, pd.DataFrame({'duration' : d['Time stop']-d['Time start'], 'onset' : value, 'trial_type' : d['NameOfCondition']})])


    # Sort dataframe by onset
    bids_df = bids_df.sort_values("onset")

    # duration and onset in miliseconds, convert to seconds
    bids_df['onset'] = bids_df['onset']/1000
    bids_df['duration'] = bids_df['duration']/1000

    return bids_df





In [3]:
# DEBUG

# dataset Raw to BIDS conversion matrix
conversion_table_parts = {'AlexandreSayal':'01',
                    }

# functional runs avaialble in the dataset
conversion_table_fruns = {'run-loc-data':'faceexplocalizer', }

In [4]:
# dataset Raw to BIDS conversion matrix
conversion_table_parts = {'AlexandreSayal':'01',
                    'GabrielCosta':'02',
                    'GilbertoSilva':'03',
                    'JoaoDuarte':'04',
                    'JoaoSebastiao':'05',
                    'MarcoLeal':'06',
                    'MarioRibeiro':'07',
                    'MiguelBernardes':'08',
                    'PatriciaSantos':'09',
                    'RicardoCouceiro':'10'}

# functional runs available in the dataset
conversion_table_fruns = {'run-loc-data':'faceexplocalizer', 
                          'run-fv-data':'visualfeedback',
                          'run-fa-data':'auditoryfeedback', 
                          'run-trans-data':'nofeedback'}



# Path to files

path_bidsdata represents the folder with the bids dataset
layout is the BIDS tree structure

path_rawdata represents the folder with the raw data


In [5]:
# BIDS dataset path
path_bidsdata = 'F:\BIDS_face-expr-nf'
path_bidsdata ='/Users/brunoleitao/Documents/Projects/TargetingFacialProcessingNeurofeedback'

# BIDS layout
layout = BIDSLayout(path_bidsdata)


In [6]:
# Raw data path
path_rawdata = '/Volumes/Elements/DATA_Controls'

# Convert .prt to .tsv

Get .prt files using the raw data structure, read files using bvbabel, and save .tsv files in the BIDS dataset.


In [7]:
# Iterate conversion table

for subject, subject_id in conversion_table_parts.items():
    print('Processing subject: ', subject, 'with id: ', subject_id)

    # Iterate functional runs
    for run, bids_run in conversion_table_fruns.items():

        # Get .prt file 
        path_prtfile = os.path.join(path_rawdata, subject, run,'PROJECT', 'ANALYSIS')

        # Find file with extension .prt in path_prtfile
        file_prt = [f for f in os.listdir(path_prtfile) if f.endswith('.prt')][0]
        print('prt file to convert: ', file_prt)
        # Fullfilename.
        FILE = os.path.join(path_prtfile, file_prt)


        # -- Copy file to ./prt-data folder. --
        os.system('cp ' + FILE + ' ../prt-data/')
        FILE = os.path.join('../prt-data', file_prt)
        
        # If last line contains string 'ResponseConditions' remove it.
        with open(FILE, 'r') as f:
            lines = f.readlines()
            if 'ResponseConditions' in lines[-1]:
                lines = lines[:-1]
        with open(FILE, 'w') as f:
            f.writelines(lines)


        # --- Convert .prt to .tsv. ---
        bids_df = convert_prt_to_tsv(FILE)
        

        output_fn = os.path.join('../prt-data/', 'sub-' + subject_id + '_ses-01_task-' + bids_run + '_run-01_events.tsv')

        # --- Save dataframe to .tsv file. ---
        bids_df.to_csv(output_fn, sep='\t', index=False)

        # -- Copy file to ./prt-data folder. --
        os.system('cp ' + output_fn + ' ' + os.path.join(path_bidsdata, 'sub-' + subject_id,'ses-01', 'func'))
    




Processing subject:  AlexandreSayal with id:  01
prt file to convert:  Dynamic_expressions_protocol_15_16-2_4_2015.prt

PRT header
   FileVersion : 2
   ResolutionOfTime : msec
   Experiment : protocol_avatars_static_images
   BackgroundColor : 0 0 0
   TextColor : 255 255 202
   TimeCourseColor : 255 255 255
   TimeCourseThick : 3
   ReferenceFuncColor : 192 192 192
   ReferenceFuncThick : 50
   NrOfConditions : 5

PRT data
   NameOfCondition : Neutral
   NrOfOccurances : 8
   Time start : [ 16000  64000  80000 136000 184000 224000 248000 296000]
   Time stop : [ 24000  72000  88000 144000 192000 232000 256000 304000]
   Color : [  0 128 255]

   NameOfCondition : Happy
   NrOfOccurances : 8
   Time start : [ 32000  72000 104000 120000 192000 200000 264000 304000]
   Time stop : [ 40000  80000 112000 128000 200000 208000 272000 312000]
   Color : [  0 255 255]

   NameOfCondition : Sad
   NrOfOccurances : 8
   Time start : [  8000  40000  88000 144000 176000 216000 256000 312000]
   T